# Part 2 - Mapping Yelp Search Results

## Obective

- For this CodeAlong, we will be working with the Yelp API results from last class. 
- You will load in the .csv.gz of your yelp results and prepare the data for visualization.
- You will use Plotly Express to create an interactive map with all of the results.

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- [Advanced Transformations with Pandas - Part 1](https://login.codingdojo.com/m/376/12529/88086)
- [Advanced Transformations with Pandas - Part 2](https://login.codingdojo.com/m/376/12529/88088)

### Goal

- We want to create a map with every restaurant plotted as a scatter plot with detailed information that appears when we hover over a business
- We will use plotly express's `px.scatter_mapbox` function to accomplish this.
    - https://plotly.com/python/scattermapbox/
    
    - We will need a Mapbox API token for some of the options:
        - https://studio.mapbox.com/
    

# Loading Data from Part 1

In [1]:
## Plotly is not included in your dojo-env
!pip install plotly

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json

## importing plotly 
import plotly.express as px

In [2]:
## Load in csv.gz
pismo_df = pd.read_csv('Data/Pismo Beach-pizza.csv.gz')
pismo_df.head(3)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,Km1aQHFPP7VwvFimVByPgg,dels-pizzeria-pismo-beach,Del's Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/dEgnqd...,False,https://www.yelp.com/biz/dels-pizzeria-pismo-b...,499,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 35.14405, 'longitude': -120.6429}","['pickup', 'delivery']",$$,"{'address1': '1101 Price St', 'address2': '', ...",1.805773e+10,(805) 773-4438,456.832734
1,UCH_lRmCHWBqzb0zs3Hbvg,pizmo-cafe-pismo-beach,PizMo Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/rno5HY...,False,https://www.yelp.com/biz/pizmo-cafe-pismo-beac...,458,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 35.1405040480287, 'longitude': -1...",['delivery'],$$,"{'address1': '270 Pomeroy Ave', 'address2': ''...",1.805556e+10,(805) 556-0347,488.774648
2,vxFH7N_Ntxg-pagIIWM75g,beach-pizza-grover-beach,Beach Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/aCoCRY...,False,https://www.yelp.com/biz/beach-pizza-grover-be...,138,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 35.12111, 'longitude': -120.61964}","['pickup', 'delivery']",NaN,"{'address1': '960 W Grand Ave', 'address2': No...",1.805474e+10,(805) 474-4499,2999.014031


## Required Preprocessing 

- 1. We need to get the latitude and longitude for each business as separate columns.
- We also want to be able to show the restaurants:
    - name,
    - price range
    - address
    - and if they do delivery or takeout.

### Separating Latitude and Longitude

In [3]:
## use .apply pd.Series to convert a dict to columns
pismo_df['coordinates'].apply(pd.Series)

,0
0,"{'latitude': 35.14405, 'longitude': -120.6429}"
1,"{'latitude': 35.1405040480287, 'longitude': -1..."
2,"{'latitude': 35.12111, 'longitude': -120.61964}"
3,"{'latitude': 35.1351676233652, 'longitude': -1..."
4,"{'latitude': 35.14256, 'longitude': -120.64055}"
...,...
83,"{'latitude': 35.12111, 'longitude': -120.61964}"
84,"{'latitude': 35.13642, 'longitude': -120.62505}"
85,"{'latitude': 35.2153483, 'longitude': -120.614..."
86,"{'latitude': 35.131459, 'longitude': -120.605904}"


- Why didn't that work???

In [4]:
## slice out a single test coordinate
test_coord = pismo_df.loc[0, 'coordinates']
test_coord

"{'latitude': 35.14405, 'longitude': -120.6429}"

- Its not a dictionary anymore!!! WTF??
    - CSV files cant store iterables (lists, dictionaries) so they get converted to strings.

### Fixing the String-Dictionaries

- The json module has another version of load and dump called `json.loads` and `json.dumps`
    - These are designed to process STRINGS instead of files. 
    
- If we use `json.loads` we can convert our string dictionary into an actual dictionary. 

In [8]:
## Use json.loads on the test coordinate
json.loads(test_coord)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

- JSON requires double quotes!

In [5]:
## replace single ' with " 
test_coord = test_coord.replace("'", '"')
test_coord

'{"latitude": 35.14405, "longitude": -120.6429}'

In [6]:
## Use json.loads on the test coordinate, again
json.loads(test_coord)

{'latitude': 35.14405, 'longitude': -120.6429}

### Now, how can we apply this same process to the entire column??

In [7]:
## replace ' with " (entire column)
pismo_df['coordinates'] = pismo_df['coordinates'].str.replace("'",'"')
## apply json.loads
pismo_df['coordinates'] = pismo_df['coordinates'].apply(json.loads)

In [8]:
## slice out a single test coordinate
test_coord = pismo_df.loc[2, 'coordinates']
test_coord

{'latitude': 35.12111, 'longitude': -120.61964}

### Using Apply with pd.Series to convert a dictionary column into multiple columns

In [9]:
## use .apply pd.Series to convert a dict to columns
pismo_df['coordinates'].apply(pd.Series)

,latitude,longitude
0,35.144050,-120.642900
1,35.140504,-120.641385
2,35.121110,-120.619640
3,35.135168,-120.624588
4,35.142560,-120.640550
...,...,...
83,35.121110,-120.619640
84,35.136420,-120.625050
85,35.215348,-120.614705
86,35.131459,-120.605904


In [10]:
## Concatenate the 2 new columns and drop the original.
pismo_df = pd.concat([pismo_df, pismo_df['coordinates'].apply(pd.Series)], axis = 1)
pismo_df = pismo_df.drop(columns = ['coordinates'])
pismo_df.head(3)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude
0,Km1aQHFPP7VwvFimVByPgg,dels-pizzeria-pismo-beach,Del's Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/dEgnqd...,False,https://www.yelp.com/biz/dels-pizzeria-pismo-b...,499,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"['pickup', 'delivery']",$$,"{'address1': '1101 Price St', 'address2': '', ...",1.805773e+10,(805) 773-4438,456.832734,35.144050,-120.642900
1,UCH_lRmCHWBqzb0zs3Hbvg,pizmo-cafe-pismo-beach,PizMo Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/rno5HY...,False,https://www.yelp.com/biz/pizmo-cafe-pismo-beac...,458,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],$$,"{'address1': '270 Pomeroy Ave', 'address2': ''...",1.805556e+10,(805) 556-0347,488.774648,35.140504,-120.641385
2,vxFH7N_Ntxg-pagIIWM75g,beach-pizza-grover-beach,Beach Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/aCoCRY...,False,https://www.yelp.com/biz/beach-pizza-grover-be...,138,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['pickup', 'delivery']",NaN,"{'address1': '960 W Grand Ave', 'address2': No...",1.805474e+10,(805) 474-4499,2999.014031,35.121110,-120.619640


## Creating a Simple Map

### Register for MapBox API

Mapbox API: https://www.mapbox.com/

In [13]:
## Load in mapbox api credentials from .secret
with open('/Users/tyler/.secret/mapbox.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client_id', 'token'])

- Use the plotly express `set_maptbox_acccess_token` function

In [14]:
## set mapbox token
px.set_mapbox_access_token(login['token'])

In [16]:
## use scatter_mapbox for M.V.P map
px.scatter_mapbox(pismo_df, lat='latitude', lon='longitude', mapbox_style = 'open-street-map')

### Adding Hover Data

- We want to show the restaurants:
    - name
    - price range
    - address
    - and if they do delivery or takeout.
    
    
- We can use the `hover_name` and `hover_data` arguments for `px.scatter_mapbox` to add this info!

In [18]:
## add hover_name (name) and hover_data for price,rating,location
px.scatter_mapbox(pismo_df, lat='latitude', lon='longitude', mapbox_style = 'open-street-map', 
                  hover_name='name', hover_data=['price', 'rating', 'location'])

### Fixing the Location Column

In [20]:
## slice out a test address
test_addr = pismo_df.loc[0, 'location']
test_addr

"{'address1': '1101 Price St', 'address2': '', 'address3': '', 'city': 'Pismo Beach', 'zip_code': '93449', 'country': 'US', 'state': 'CA', 'display_address': ['1101 Price St', 'Pismo Beach, CA 93449']}"

> Also a string-dictionary...

In [21]:
## replace ' with "
pismo_df['location'] = pismo_df['location'].str.replace("'", '"')
pismo_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude
0,Km1aQHFPP7VwvFimVByPgg,dels-pizzeria-pismo-beach,Del's Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/dEgnqd...,False,https://www.yelp.com/biz/dels-pizzeria-pismo-b...,499,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"['pickup', 'delivery']",$$,"{""address1"": ""1101 Price St"", ""address2"": """", ...",1.805773e+10,(805) 773-4438,456.832734,35.144050,-120.642900
1,UCH_lRmCHWBqzb0zs3Hbvg,pizmo-cafe-pismo-beach,PizMo Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/rno5HY...,False,https://www.yelp.com/biz/pizmo-cafe-pismo-beac...,458,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],$$,"{""address1"": ""270 Pomeroy Ave"", ""address2"": """"...",1.805556e+10,(805) 556-0347,488.774648,35.140504,-120.641385
2,vxFH7N_Ntxg-pagIIWM75g,beach-pizza-grover-beach,Beach Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/aCoCRY...,False,https://www.yelp.com/biz/beach-pizza-grover-be...,138,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['pickup', 'delivery']",NaN,"{""address1"": ""960 W Grand Ave"", ""address2"": No...",1.805474e+10,(805) 474-4499,2999.014031,35.121110,-120.619640
3,4fAGqkTIOMmkXyylyQXhBw,ginos-pizza-pismo-beach,Gino's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/kPxejj...,False,https://www.yelp.com/biz/ginos-pizza-pismo-bea...,361,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"['pickup', 'delivery']",$$,"{""address1"": ""539 Five Cities Dr"", ""address2"":...",1.805556e+10,(805) 556-0939,1533.749594,35.135168,-120.624588
4,kbCBKe2CcNpjy61KoZ907Q,giuseppes-express-pismo-beach,Giuseppe's Express,https://s3-media2.fl.yelpcdn.com/bphoto/goUVCf...,False,https://www.yelp.com/biz/giuseppes-express-pis...,378,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,['delivery'],$$,"{""address1"": ""800 Price St"", ""address2"": """", ""...",1.805773e+10,(805) 773-2873,284.396328,35.142560,-120.640550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Ny2Qj6ywKrHsuqBcvg4kww,julias-juice-bar-grover-beach-2,Julia's Juice Bar,https://s3-media2.fl.yelpcdn.com/bphoto/Duwdva...,False,https://www.yelp.com/biz/julias-juice-bar-grov...,115,"[{'alias': 'juicebars', 'title': 'Juice Bars &...",4.5,[],$$,"{""address1"": ""960 W Grand Ave"", ""address2"": ""S...",1.805481e+10,(805) 481-4100,2999.014031,35.121110,-120.619640
84,pEcfqzB3-Q7lhJpitGWhLg,cocos-bakery-restaurant-pismo-beach-3,Coco's Bakery Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/qc0hm2...,False,https://www.yelp.com/biz/cocos-bakery-restaura...,140,"[{'alias': 'tradamerican', 'title': 'American ...",3.5,['delivery'],$$,"{""address1"": ""411 5 Cities Dr"", ""address2"": """"...",1.805773e+10,(805) 773-3240,1416.532006,35.136420,-120.625050
85,206Aj3TkUeCvFJojHFXinQ,biddle-ranch-vineyard-san-luis-obispo-2,Biddle Ranch Vineyard,https://s3-media2.fl.yelpcdn.com/bphoto/Lr6AiN...,False,https://www.yelp.com/biz/biddle-ranch-vineyard...,88,"[{'alias': 'wineries', 'title': 'Wineries'}]",4.5,[],$$,"{""address1"": ""2050 Biddle Ranch Rd"", ""address2...",1.805543e+10,(805) 543-2399,8231.417719,35.215348,-120.614705
86,pishOto_0wHMxiiUJ5NV_A,taco-bell-pismo-beach,Taco Bell,https://s3-media4.fl.yelpcdn.com/bphoto/pKWGnY...,False,https://www.yelp.com/biz/taco-bell-pismo-beach...,59,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",2.5,['delivery'],$,"{""address1"": ""867 Oak Park Blvd"", ""address2"": ...",1.805474e+10,(805) 474-4141,3205.195625,35.131459,-120.605904


In [22]:
## apply json.loads
pismo_df['location'] = pismo_df['location'].apply(json.loads)
pismo_df

JSONDecodeError: Expecting value: line 1 column 45 (char 44)

> Ruh roh....

- Hmm, let's slice out a test_address again and let's write a function to accomplish this instead.
    - We can use try and except in our function to get around the errors.

### Fixing Addresses - with a custom function


In [23]:
## slice out test address 
test_addr = pismo_df.loc[0, 'location']
test_addr

'{"address1": "1101 Price St", "address2": "", "address3": "", "city": "Pismo Beach", "zip_code": "93449", "country": "US", "state": "CA", "display_address": ["1101 Price St", "Pismo Beach, CA 93449"]}'

In [24]:
## write a function to just run json.loads on the address
def fix_address(test_addr):
    try: 
        return json.loads(test_addr)
    except:
        return 'Error'

In [25]:
## test applying our function
pismo_df['location'].apply(fix_address)

0     {'address1': '1101 Price St', 'address2': '', ...
1     {'address1': '270 Pomeroy Ave', 'address2': ''...
2                                                 Error
3                                                 Error
4     {'address1': '800 Price St', 'address2': '', '...
                            ...                        
83    {'address1': '960 W Grand Ave', 'address2': 'S...
84    {'address1': '411 5 Cities Dr', 'address2': ''...
85    {'address1': '2050 Biddle Ranch Rd', 'address2...
86    {'address1': '867 Oak Park Blvd', 'address2': ...
87    {'address1': '1500 W Grand Ave', 'address2': '...
Name: location, Length: 88, dtype: object

- It worked! Now let's save this as a new column (display_location),
and then let's investigate the businesses that had an "ERROR".

In [26]:
### save a new display_location column using our function
pismo_df['display_location'] = pismo_df['location'].apply(fix_address)

In [31]:
## filter for businesses with display_location == "ERROR"
errors = pismo_df['display_location'] =='Error'
pismo_df[errors]

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude,display_location
2,vxFH7N_Ntxg-pagIIWM75g,beach-pizza-grover-beach,Beach Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/aCoCRY...,False,https://www.yelp.com/biz/beach-pizza-grover-be...,138,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['pickup', 'delivery']",NaN,"{""address1"": ""960 W Grand Ave"", ""address2"": No...",1.805474e+10,(805) 474-4499,2999.014031,35.121110,-120.619640,Error
3,4fAGqkTIOMmkXyylyQXhBw,ginos-pizza-pismo-beach,Gino's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/kPxejj...,False,https://www.yelp.com/biz/ginos-pizza-pismo-bea...,361,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"['pickup', 'delivery']",$$,"{""address1"": ""539 Five Cities Dr"", ""address2"":...",1.805556e+10,(805) 556-0939,1533.749594,35.135168,-120.624588,Error
5,dWNxwCf8R_SDJf63E4pfAw,me-n-ed-s-pizzeria-and-craft-house-pismo-beach-2,Me n Ed’s Pizzeria and Craft House,https://s3-media2.fl.yelpcdn.com/bphoto/7fpkEX...,False,https://www.yelp.com/biz/me-n-ed-s-pizzeria-an...,65,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,[],NaN,"{""address1"": ""750 Price St"", ""address2"": None,...",1.805556e+10,(805) 556-3636,281.184189,35.142100,-120.640040,Error
7,EqLSwFmOJ6kosvRFELw70w,palo-mesa-wood-fired-arroyo-grande,Palo Mesa Wood Fired,https://s3-media4.fl.yelpcdn.com/bphoto/qS_y0w...,False,https://www.yelp.com/biz/palo-mesa-wood-fired-...,184,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",4.5,['delivery'],$$,"{""address1"": ""150 West Branch St"", ""address2"":...",1.805905e+10,(805) 904-6807,5651.900210,35.122830,-120.581130,Error
9,5kwOxAZw2Ychp2xI37R1QA,blast-and-brew-pismo-beach-2,Blast & Brew,https://s3-media2.fl.yelpcdn.com/bphoto/0I65uI...,False,https://www.yelp.com/biz/blast-and-brew-pismo-...,363,"[{'alias': 'newamerican', 'title': 'American (...",3.5,['delivery'],$$,"{""address1"": ""2131 Price St"", ""address2"": None...",1.805296e+10,(805) 295-5151,1114.239537,35.148269,-120.648726,Error
10,UMM2irwWRWg_jbh2vbTZjg,grover-beach-sourdough-grover-beach-2,Grover Beach Sourdough,https://s3-media2.fl.yelpcdn.com/bphoto/JCGJ57...,False,https://www.yelp.com/biz/grover-beach-sourdoug...,155,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",5.0,['delivery'],$,"{""address1"": ""236 W Grand Ave"", ""address2"": """"...",1.805668e+10,(805) 668-2256,2609.226761,35.121747,-120.628579,Error
12,YOIdczoG4nNzJ2DMFgg1qg,pismo-s-world-famous-pizza-and-chowder-house-p...,Pismo’s World Famous Pizza & Chowder House,https://s3-media3.fl.yelpcdn.com/bphoto/vQynNK...,False,https://www.yelp.com/biz/pismo-s-world-famous-...,59,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,[],$$,"{""address1"": ""690 Dolliver St"", ""address2"": """"...",NaN,NaN,452.583803,35.140673,-120.640953,Error
14,SkOgvn62-hID__rsofeDeQ,klondike-pizza-arroyo-grande,Klondike Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/EO7Z-O...,False,https://www.yelp.com/biz/klondike-pizza-arroyo...,400,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],$$,"{""address1"": ""104 Bridge St"", ""address2"": None...",1.805482e+10,(805) 481-5288,5813.018552,35.122914,-120.579206,Error
26,QjBOzIlFo9rxzlQwKbqCKw,blaze-pizza-arroyo-grande-2,BLAZE PIZZA,https://s3-media2.fl.yelpcdn.com/bphoto/kEi-P0...,False,https://www.yelp.com/biz/blaze-pizza-arroyo-gr...,177,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",4.0,"['pickup', 'delivery']",$,"{""address1"": ""1168 W Branch St"", ""address2"": N...",1.805825e+10,(805) 825-3750,4046.329607,35.128611,-120.597323,Error
28,MNicp-QHlNR2ZfPum0OByg,jimmy-d-s-spaghetti-joint-grover-beach,Jimmy D’s Spaghetti Joint,https://s3-media2.fl.yelpcdn.com/bphoto/s9sSxp...,False,https://www.yelp.com/biz/jimmy-d-s-spaghetti-j...,75,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,[],$$,"{""address1"": ""1301 W Grand Ave"", ""address2"": ""...",1.805905e+10,(805) 904-6898,3219.667333,35.121513,-120.615124,Err

In [34]:
## slice out a new test address and inspect
test_addr = pismo_df.loc[73, 'location']
test_addr

'{"address1": "967 W Grand Ave", "address2": "", "address3": None, "city": "Grover Beach", "zip_code": "93433", "country": "US", "state": "CA", "display_address": ["967 W Grand Ave", "Grover Beach, CA 93433"]}'

> After some more investigation, we would find a few issues with these "ERROR" rows.
1. They contained None.
2. They contained an apostrophe in the name.
3. ...?

### Possible Fixes (if we care to/have the time)


- Use Regular Expressions to find an fix the display addresses with "'" in them
- Use string split to split on the word display address.
    - Then use string methods to clean up

### Moving Forward without those rows (for now)

In [35]:
## remove any rows where display_location == 'ERROR'
pismo_df = pismo_df.loc[pismo_df['display_location']!='Error']
pismo_df.head(4)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude,display_location
0,Km1aQHFPP7VwvFimVByPgg,dels-pizzeria-pismo-beach,Del's Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/dEgnqd...,False,https://www.yelp.com/biz/dels-pizzeria-pismo-b...,499,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"['pickup', 'delivery']",$$,"{""address1"": ""1101 Price St"", ""address2"": """", ...",1.805773e+10,(805) 773-4438,456.832734,35.144050,-120.642900,"{'address1': '1101 Price St', 'address2': '', ..."
1,UCH_lRmCHWBqzb0zs3Hbvg,pizmo-cafe-pismo-beach,PizMo Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/rno5HY...,False,https://www.yelp.com/biz/pizmo-cafe-pismo-beac...,458,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],$$,"{""address1"": ""270 Pomeroy Ave"", ""address2"": """"...",1.805556e+10,(805) 556-0347,488.774648,35.140504,-120.641385,"{'address1': '270 Pomeroy Ave', 'address2': ''..."
4,kbCBKe2CcNpjy61KoZ907Q,giuseppes-express-pismo-beach,Giuseppe's Express,https://s3-media2.fl.yelpcdn.com/bphoto/goUVCf...,False,https://www.yelp.com/biz/giuseppes-express-pis...,378,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,['delivery'],$$,"{""address1"": ""800 Price St"", ""address2"": """", ""...",1.805773e+10,(805) 773-2873,284.396328,35.142560,-120.640550,"{'address1': '800 Price St', 'address2': '', '..."
6,NpngL0FEuj9OxaXPzqhNYg,rosas-italian-restaurant-pismo-beach-3,Rosa's Italian Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/mGH8L4...,False,https://www.yelp.com/biz/rosas-italian-restaur...,1208,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,['delivery'],$$,"{""address1"": ""491 Price St"", ""address2"": """", ""...",1.805773e+10,(805) 773-0551,416.138931,35.140170,-120.638440,"{'address1': '491 Price St', 'address2': '', '..."


- We want the "display_address" key from the "display_location" dictionaries.
- We could use a .apply and a lamda to slice out the desired key.

In [36]:
## use apply and lambda to slice correct key
pismo_df['display_location'] = pismo_df['display_location'].apply(lambda x: x['display_address'])
pismo_df.head(3)

C:\Users\tyler\AppData\Local\Temp\ipykernel_12396\2693214443.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude,display_location
0,Km1aQHFPP7VwvFimVByPgg,dels-pizzeria-pismo-beach,Del's Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/dEgnqd...,False,https://www.yelp.com/biz/dels-pizzeria-pismo-b...,499,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"['pickup', 'delivery']",$$,"{""address1"": ""1101 Price St"", ""address2"": """", ...",1.805773e+10,(805) 773-4438,456.832734,35.144050,-120.642900,"[1101 Price St, Pismo Beach, CA 93449]"
1,UCH_lRmCHWBqzb0zs3Hbvg,pizmo-cafe-pismo-beach,PizMo Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/rno5HY...,False,https://www.yelp.com/biz/pizmo-cafe-pismo-beac...,458,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],$$,"{""address1"": ""270 Pomeroy Ave"", ""address2"": """"...",1.805556e+10,(805) 556-0347,488.774648,35.140504,-120.641385,"[270 Pomeroy Ave, Pismo Beach, CA 93449]"
4,kbCBKe2CcNpjy61KoZ907Q,giuseppes-express-pismo-beach,Giuseppe's Express,https://s3-media2.fl.yelpcdn.com/bphoto/goUVCf...,False,https://www.yelp.com/biz/giuseppes-express-pis...,378,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,['delivery'],$$,"{""address1"": ""800 Price St"", ""address2"": """", ""...",1.805773e+10,(805) 773-2873,284.396328,35.142560,-120.640550,"[800 Price St, Pismo Beach, CA 93449]"


- Almost done! We want to convert display_address to a string instead a list of strings.
- We can use the string method .join to do so!

In [38]:
## slice out test address 
test_addr = pismo_df.loc[0, 'display_location']
test_addr

['1101 Price St', 'Pismo Beach, CA 93449']

In [39]:
## test using .join with a "\n"
'\n'.join(test_addr)

'1101 Price St\nPismo Beach, CA 93449'

In [41]:
## apply the join to every row with a lambda
pismo_df['address'] = pismo_df['display_location'].apply(lambda x: '\n'.join(x))
pismo_df

C:\Users\tyler\AppData\Local\Temp\ipykernel_12396\4207290606.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude,display_location,address
0,Km1aQHFPP7VwvFimVByPgg,dels-pizzeria-pismo-beach,Del's Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/dEgnqd...,False,https://www.yelp.com/biz/dels-pizzeria-pismo-b...,499,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"['pickup', 'delivery']",$$,"{""address1"": ""1101 Price St"", ""address2"": """", ...",1.805773e+10,(805) 773-4438,456.832734,35.144050,-120.642900,"[1101 Price St, Pismo Beach, CA 93449]","1101 Price St\nPismo Beach, CA 93449"
1,UCH_lRmCHWBqzb0zs3Hbvg,pizmo-cafe-pismo-beach,PizMo Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/rno5HY...,False,https://www.yelp.com/biz/pizmo-cafe-pismo-beac...,458,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],$$,"{""address1"": ""270 Pomeroy Ave"", ""address2"": """"...",1.805556e+10,(805) 556-0347,488.774648,35.140504,-120.641385,"[270 Pomeroy Ave, Pismo Beach, CA 93449]","270 Pomeroy Ave\nPismo Beach, CA 93449"
4,kbCBKe2CcNpjy61KoZ907Q,giuseppes-express-pismo-beach,Giuseppe's Express,https://s3-media2.fl.yelpcdn.com/bphoto/goUVCf...,False,https://www.yelp.com/biz/giuseppes-express-pis...,378,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,['delivery'],$$,"{""address1"": ""800 Price St"", ""address2"": """", ""...",1.805773e+10,(805) 773-2873,284.396328,35.142560,-120.640550,"[800 Price St, Pismo Beach, CA 93449]","800 Price St\nPismo Beach, CA 93449"
6,NpngL0FEuj9OxaXPzqhNYg,rosas-italian-restaurant-pismo-beach-3,Rosa's Italian Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/mGH8L4...,False,https://www.yelp.com/biz/rosas-italian-restaur...,1208,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,['delivery'],$$,"{""address1"": ""491 Price St"", ""address2"": """", ""...",1.805773e+10,(805) 773-0551,416.138931,35.140170,-120.638440,"[491 Price St, Pismo Beach, CA 93449]","491 Price St\nPismo Beach, CA 93449"
8,Alnpp0cDLJ5ZsDEpzWjuWg,the-boardroom-pismo-beach-4,The Boardroom,https://s3-media1.fl.yelpcdn.com/bphoto/RXxgk7...,False,https://www.yelp.com/biz/the-boardroom-pismo-b...,279,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.0,['delivery'],$$,"{""address1"": ""160 Hinds Ave"", ""address2"": """", ...",1.805296e+10,(805) 295-6222,661.720776,35.138780,-120.641880,"[160 Hinds Ave, Pismo Beach, CA 93449]","160 Hinds Ave\nPismo Beach, CA 93449"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Ny2Qj6ywKrHsuqBcvg4kww,julias-juice-bar-grover-beach-2,Julia's Juice Bar,https://s3-media2.fl.yelpcdn.com/bphoto/Duwdva...,False,https://www.yelp.com/biz/julias-juice-bar-grov...,115,"[{'alias': 'juicebars', 'title': 'Juice Bars &...",4.5,[],$$,"{""address1"": ""960 W Grand Ave"", ""address2"": ""S...",1.805481e+10,(805) 481-4100,2999.014031,35.121110,-120.619640,"[960 W Grand Ave, Ste F, Grover Beach, CA 93433]","960 W Grand Ave\nSte F\nGrover Beach, CA 93433"
84,pEcfqzB3-Q7lhJpitGWhLg,cocos-bakery-restaurant-pismo-beach-3,Coco's Bakery Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/qc0hm2...,False,https://www.yelp.com/biz/cocos-bakery-restaura...,140,"[{'alias': 'tradamerican', 'title': 'American ...",3.5,['delivery'],$$,"{""address1"": ""411 5 Cities Dr"", ""address2"": """"...",1.805773e+10,(805) 773-3240,1416.532006,35.136420,-120.625050,"[411 5 Cities Dr, Pismo Beach, CA 93449]","411 5 Cities Dr\nPismo Beach, CA 93449"
85,206Aj3TkUeCvFJojHFXinQ,biddle-ranch-vineyard-san-luis-obispo-2,Biddle Ranch Vineyard,https://s3-media2.fl.yelpcdn.com/bphoto/Lr6AiN...,False,https://www.yelp.com/biz/biddle-ranch-vineyard...,88,"[{'alias': 'wineries', 'title': 'Wineries'}]",4.5,[],$$,"{""address1"": ""2050 Biddle Ranch Rd"", ""address2...",1.805543e+10,(805) 543-2399,8231.417719,35.215348,-120.614705,"[2050 Biddle Ranch Rd, San Luis Obispo, CA 93401]","2050 Biddle Ranch Rd\nSan Luis Obispo, CA 93401"
86,pishOto_0wHMxiiUJ5NV_A,taco-bell-pismo-

### Final Map

In [43]:
## make ourn final map and save as varaible
pfig = px.scatter_mapbox(pismo_df, lat='latitude', lon='longitude', mapbox_style = 'open-street-map', 
                  hover_name='name', hover_data=['price', 'rating', 'address'], 
                        color = 'rating', title = 'Pizza in Pismo')
pfig.show()

#### HTML Uses `<br>` instead of `\n`

In [ ]:
## remake the final address column with <br> instead 

## plot the final map

In [44]:
## use fig.write_html to save map
pfig.write_html('final_map.html')